<a href="https://colab.research.google.com/github/YasJanam/NLP_MODELS_1/blob/main/Pretrain_Methods_11/Pretrain_Methods_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **PRETRAIN**

🌞پیش آموزش مدل های بارت یکی از مهم ترین قسمت های اموزش این مدل هاست
پیش آموزش مثل این است که مدل اول زبان را یادبگیرد.سپس برای وظیفه مربوطه،مثلا خلاصه سازی یا ترجمه آموزش ببیند

🌟 پیش‌آموزش معمولاً با هدف‌های عمومی انجام میشه مثل :

 - Masked Language Modeling (MLM) → حدس زدن توکن‌های حذف‌ شده

 - Denoising Autoencoding → بازسازی متن ناقص یا بهم‌ ریخته

In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset
from transformers import AutoTokenizer
import random
import math

پیش آموزش مدل های بارت در حقیقت خودش یک آموزش است که روی یک سری دیتاست ها انجام میشود.در حقیقت چیزی که باید برای پیش آموزش آماده کنیم مدل خاص،یا ابزار آموزش خاصی نیست؛بلکه باید دیتاست مناسب پیش آموزش را آماده کنیم

🟧 __دیتاست مناسب برای پیش آموزش چه دیتاستی است ؟__

دیتاست مناسب برای پیش آموزش یک دیتاست متنی است ، به این صورت که هر نمونه از این دیتاست دو متن اصلی دارد : یکی متنی به هم ریخته و نویز دار و دیگری مرتب شده همان متن به هم ریخته و نویز دار

🟡 __شکل کلی دیتاست مناسب پیش آموزش :__    

 - __text__ ⟹ متن به هم ریخته،نویز دار همراه با جای خالی

 - __label__ ⟹ شکل صحیح و درست همان متن نویز دار


در مرحله پیش آموزش،مدل یاد میگیرد که متن به هم ریخته را درست کند

✅ __چیزهایی که مدل در مرحله پیش آموزش یاد میگیرد__

1. جمله بندی و ساختار کلی جمله ها
2. واژگان و چگونگی استفاده از آنها در جمله
3. توانایی مرتب کردن جمله به هم ریخته
4. توانایی پر کردن جای خالی در جمله
5. و در کل آشنایی با زبان مربوطه و قواعدش


پس میبینیم که این مرحله چقدر ضروری بوده است. بدون پیش آموزش مثل این است که بچه ای که صفر هست و زبان بلد نیست را یک دفعه ای ببریم و بهش خلاصه سازی یاد بدهیم

🔴 پس چیزی که بسیار مهم است آماده سازی یک دیتاست خوب است

برای ساخت دیتاست مناسب پیش آموزش نیاز به یک دیتاست متنی داریم که فقط متن داشته باشد ⟹ **"wikitext-103-v1"**

🟩 دیتاستی که میخواهیم بسازیم در کل سه ستون دارد :    
1. input_ids ⟹ توکنایز شده متن بهم ریخته
2. attention_mask ⟹ ماسک متن بهم ریخته
3. labels ⟹ توکنایز شده متن اصلی و سالم

🟥 مسیر کلی برای ساخت این دیتاست :    
1. تعریف یک سری متد برای به هم ریختن یک جمله، ایجاد جای خالی در جمله به شکل تصادفی و نویز دادن به جمله
2. استفاده از متد های بالا روی همه جملات دیتاست
3. توکنایز کردن جمله های بدست آمده از مرحله دو
4. توکنایز کردن فرم اصلی هر جمله و قرار دادن آن کنار توکنایز شده فرم به هم ریخته اش

↪ در ادامه متد های تعریف شده و شیوه به کارگیری آنها برای ساخت دیتاست دیده میشود

In [46]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
PAD_ID = tokenizer.pad_token_id
MASK_ID =  tokenizer.mask_token_id
EOS_ID = tokenizer.eos_token_id
BOS_ID = tokenizer.bos_token_id

In [4]:
text = """Artificial Intelligence (AI) is a branch of computer science that focuses on building systems capable of performing tasks that normally require human intelligence.
 These tasks include learning, reasoning, problem-solving, and understanding natural language.
  AI technologies are widely used today, from recommendation systems and chatbots to self-driving cars and medical diagnosis.
 As AI continues to develop, it raises both exciting opportunities and important ethical challenges for society."""

#### **methods**

In [5]:
# حذف خطوط خالی
def clean_lines(lines):
  out = []
  for line in lines:
    line = line.strip()
    if len(line) > 0:
      out.append(line)
  return out

In [ ]:
train_txt = clean_lines(train_txt)
val_txt = clean_lines(val_txt)

##### **split_sentences**
این متد یک متن را میگیرد و در عوض لیستی از جمله های آن متد را میدهد. در حقیقت متن را جمله جمله میکند

در ادامه مثالش را روی تسک بالا میبینیم

In [6]:
# ابزار جمله بندی خیلی ساده
def split_sentences(x):
  out = []
  s = []
  for tok in x.split():
    s.append(tok)
    if tok.endswith(('.','!','?','."',"!'","?'")):
      out.append(" ".join(s))
      s = []
  if s:
    out.append(" ".join(s))
  return out if out else [x]

In [8]:
# example
text_sncs = split_sentences(text)
text_sncs

['Artificial Intelligence (AI) is a branch of computer science that focuses on building systems capable of performing tasks that normally require human intelligence.',
 'These tasks include learning, reasoning, problem-solving, and understanding natural language.',
 'AI technologies are widely used today, from recommendation systems and chatbots to self-driving cars and medical diagnosis.',
 'As AI continues to develop, it raises both exciting opportunities and important ethical challenges for society.']

##### **simple_span_len**
این متد تنها کاری که میکند این است که یک طول به ما بر میگرداند.این طول تصادفی است و از توزیع پواسون استفاده میشود

اما این طول ها به چه دردی میخورند ؟

از این طول ها وقتی استفاده میکنیم که بخواهیم یک توالی در یک جمله را جای خالی کنیم. طول این توالی برای جای خالی کردن به کمک این متد بدست می آید، در ادامه خواهید دید


In [10]:
# نمونه گیری طول با توزیع پواسون
def simple_span_len(lam=3):
  L=0
  p = math.exp(-lam)
  F = p
  u = random.random()
  while u > F:
    L += 1
    p *= lam/L
    F += p
  return max(L,1)

In [12]:
l = simple_span_len()
l

4

##### **text_infilling_token_ids**
این متد برای جای خالی کرد یک توالی در یک جمله است

جمله ها کار میشود ids در این متد با فرم

توضیح کد :    

محاسبه یک طول تصادفی :    

    L = len(ids)

محاسبه طول دقیق طول توالی برای ماسک کردن :

(که باید جای خالی شوند ids  تعداد عدد هایی از )     

    num_to_mask = max(1,int(mask_ratio*L))

تعریف لیست ماسک شده، لیستی که در نهایت برگردانده میشود.در ابتدا این لیست   برابر جمله اصلی است:

    masked = ids[:]

تعریف مجموعه ای که شامل ایندکس هایی از جمله است که ماسک شده اند

    covered = set()

شروع فرایند ماسک کردن :    
      
    while len(covered) < num_to_mask:
      start = random.randrange(0,L)
      if start in covered:
        continue
      span_len = simple_span_len(lam)
      end = min(L,start+span_len)
      for i in range(start,end):
        if i not in covered:
          masked[i] = MASK_ID
          covered.add(i)

---
شرح فرآیند ماسک کردن :
( while  حلقه )   

فرایند ماسک کردن را به اندازه طول محاسبه شده برای توالی ماسک انجام بده:

    while len(covered) < num_to_mask:

محاسبه ایندکس تصادفی برای شروع ماسک گذاری:

      start = random.randrange(0,L)

این خط میگوید اگرایندکس شروع ماسک گذاری قبلا ماسک شده است برو و دوباره ایندکس شروع را محاسبه کن:

      if start in covered:
        continue

محاسبه ایندکس پایان ماسک گذاری
( توجه کنید که ایندکس پایان ماسک گذاری لزوما ماسک شدن به اندازه محاسبه شده را تضمین نمیکند، برای همین در حلقه شروع فرایند ماسک گذاری، تعداد ماسک شده ها رو بررسی میکنیم.)

      span_len = simple_span_len(lam)
      end = min(L,start+span_len)

حالا از ایندکس شرو تا ایندکس پایان شروع به ماسک کردن میکند، هر چه که ماسک شد  اضافه میشود covered ایندکسش به مجموعه

      for i in range(start,end):
        if i not in covered:
          masked[i] = MASK_ID
          covered.add(i)

In [14]:
def text_infilling_token_ids(ids,mask_ratio=0.3,lam=3):
  L = len(ids)
  num_to_mask = max(1,int(mask_ratio*L))
  masked = ids[:]
  covered = set()
  while len(covered) < num_to_mask:
    start = random.randrange(0,L)
    if start in covered:
      continue
    span_len = simple_span_len(lam)
    end = min(L,start+span_len)
    for i in range(start,end):
      if i not in covered:
        masked[i] = MASK_ID
        covered.add(i)
  return masked

In [22]:
# example
masked_txt =  text_infilling_token_ids(tokenizer(text)["input_ids"])
print(tokenizer.decode(masked_txt))

<s><mask>ificial Intelligence (AI) is a branch of computer science that focuses on building systems capable<mask> performing<mask><mask><mask> require human<mask>.<mask><mask><mask> include learning, reasoning, problem-solving, and understanding natural language.
  AI technologies are widely used today,<mask> recommendation systems and chatbots to<mask><mask><mask><mask><mask><mask><mask><mask><mask><mask><mask><mask><mask><mask><mask><mask> raises both exciting opportunities and important ethical challenges for society<mask></s>


##### **sentence_permute**

به هم ریختن جمله ها

In [33]:
def sentence_permute(text):
  sents = split_sentences(text)
  random.shuffle(sents)
  return " ".join(sents)

In [45]:
# example
sent_per = sentence_permute(text)
sent_per

'AI technologies are widely used today, from recommendation systems and chatbots to self-driving cars and medical diagnosis. Artificial Intelligence (AI) is a branch of computer science that focuses on building systems capable of performing tasks that normally require human intelligence. These tasks include learning, reasoning, problem-solving, and understanding natural language. As AI continues to develop, it raises both exciting opportunities and important ethical challenges for society.'

##### **apply_noise**
این متد یک تسکت میگیرد. جمله های تسکت را به هم میریزد. روی تسکت نویز اعمال میکند

 ست میکند max_len سپس طول متن نویزی را به اندازه حداکثر طول یا همان

 ماسک توجه هم میسازد که نشان دهد کدام بخش هایی از جمله، واقعا قسمتی از جمله است، و کدام بخش هایی از جمله پدینگ هست

 در این متد از متد های تعریف شده قبلی استفاده شده است

 است pack تنها قسمت جدید این متد، متد


 🟪 **pack** ⟶

مشخص کردن اول و اخر جمله

    arr = [BOS_ID] + arr[:max_len-2] + [EOS_ID]

ماسک توجه : تاجایی که طول جمله است، 1 بزار

    attn = [1]*len(arr)

در تکه کد زیر طول جمله را به اندازه طول حداکثری میکنیم.اگر طول جمله کم بود، به ان اضافه میکنیم (padding ):    

    if len(arr) < max_len:
      pad_len = max_len - len(arr)
      arr += [PAD_ID]*pad_len
      attn += [0]*pad_len

ماسک توجه برای قسمت های پد شده برابر صفر است:

    attn += [0]*pad_len

In [53]:
MAX_INPUT = 512
MAX_TARGET = 128

def apply_noise(text,do_sentperm=True,mask_ratio = 0.3,lam=3):

  # میتوان تعیین کرد که جمله ها به هم بریزند یا نه
  if do_sentperm:
    text = sentence_permute(text)

  # ids -> این مربوط به جمله اصلی است.یعنی شکل صحیح و درست جمله
  ids = tokenizer(text,truncation=True,max_length=MAX_INPUT,add_special_tokens=False)["input_ids"]
  if len(ids) == 0:
    return None

  noisy_ids = text_infilling_token_ids(ids,mask_ratio=mask_ratio,lam=lam)


  def pack(arr,max_len):
    arr = [BOS_ID] + arr[:max_len-2] + [EOS_ID]
    attn = [1]*len(arr)
    if len(arr) < max_len:
      pad_len = max_len - len(arr)
      arr += [PAD_ID]*pad_len
      attn += [0]*pad_len
    return arr, attn

  noisy_imp, noisy_attn = pack(noisy_ids,MAX_INPUT)
  clean_tgt, _  = pack(ids,MAX_TARGET)

  return {
      "input_ids":noisy_imp, # شکل نویزی جمله
      "attention_mask":noisy_attn, # ماسک توجه شکل نویزی جمله
      "labels":clean_tgt  # شکل صحیح جمله
  }

In [54]:
nois_txt = apply_noise(text)
print(tokenizer.decode(nois_txt["input_ids"]))

<s>Artificial Intelligence (<mask><mask> is a branch of computer science that<mask><mask><mask><mask><mask><mask><mask> tasks that normally<mask><mask> intelligence<mask><mask><mask><mask><mask><mask> reasoning<mask><mask>-solving, and understanding<mask><mask>.<mask><mask><mask> to develop, it raises both exciting opportunities and important ethical challenges for society. AI technologies are widely used today, from recommendation systems and chatbots to self-driving cars and medical diagnosis<mask></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

##### **make_tensor_dataset**
این متد دیتاست را میگیرد و نویز را به تکست های دیتاست اعمال میکند.خروجی را به بر میگرداند Dataset فرم

In [55]:
def make_tensor_dataset(texts, n_limits=None):
  rows = []
  count = 0
  for t in texts:
    ex = apply_noise(t,do_sentperm=True,mask_ratio=0.3,lam=3)
    if ex is not None:
      rows.append(ex)
      count += 1
      if n_limits is not None and count >= n_limits:
          break

  input_ids = torch.tensor([r["input_ids"] for r in rows], dtype=torch.long)
  attention_mask = torch.tensor([r["attention_mask"] for r in rows], dtype=torch.long)
  labels = torch.tensor([r["labels"] for r in rows], dtype=torch.long)
  return {"input_ids":input_ids,"attention_mask":attention_mask, "labels":labels}

In [56]:
data = make_tensor_dataset(text)
data

{'input_ids': tensor([[    0, 50264,     2,  ...,     1,     1,     1],
         [    0, 50264,     2,  ...,     1,     1,     1],
         [    0, 50264,     2,  ...,     1,     1,     1],
         ...,
         [    0, 50264,     2,  ...,     1,     1,     1],
         [    0, 50264,     2,  ...,     1,     1,     1],
         [    0, 50264,     2,  ...,     1,     1,     1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[  0, 250,   2,  ...,   1,   1,   1],
         [  0, 338,   2,  ...,   1,   1,   1],
         [  0,  90,   2,  ...,   1,   1,   1],
         ...,
         [  0,  90,   2,  ...,   1,   1,   1],
         [  0, 219,   2,  ...,   1,   1,   1],
         [  0,   4,   2,  ...,   1,   1,   1]])}